In [1]:
cd projects/snake_tree/notebooks

/work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/snake_tree/notebooks


In [2]:
%load_ext autoreload
%autoreload 2
import glob
import os
import numpy as np
import sys
sys.path.append( '../src/')
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from compile_treeset import compile_folder , compare_treesets
import plotly.express as px
import plotly.figure_factory as ff
import plotly.express as px
from scipy.stats import wilcoxon, ks_2samp , describe
import treescore

In [3]:
sample_hogs = glob.glob('../OMA_data_unfiltered/OMA_data/LUCA/*/')
sample_hogs = sample_hogs[0:10]

In [4]:
print(sample_hogs)
#make a test directory and copy the files there

if not os.path.exists('test'):
    os.makedirs('test')
    for hog in sample_hogs:
        os.system('cp -r '+hog+' test/')
    print('copied files')
else:
    #clear files
    os.system('rm -r test')
    os.makedirs('test')
    for hog in sample_hogs:
        os.system('cp -r '+hog+' test/')
    print('copied files')
    

['../OMA_data_unfiltered/OMA_data/LUCA/HOG1077/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG748/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG458/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG411/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG569/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG1118/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG423/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG600/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG475/', '../OMA_data_unfiltered/OMA_data/LUCA/HOG1231/']
copied files


In [5]:
testhogs = glob.glob('test/*/')
print(testhogs)
sample1 = testhogs[9]
samplefiles = glob.glob(sample1+'*')
print(samplefiles)

['test/HOG1077/', 'test/HOG748/', 'test/HOG458/', 'test/HOG411/', 'test/HOG569/', 'test/HOG1118/', 'test/HOG423/', 'test/HOG600/', 'test/HOG475/', 'test/HOG1231/']
['test/HOG1231/3di_xaln_tree.nwk', 'test/HOG1231/fident_1_raw_struct_tree.PP.nwk.rooted', 'test/HOG1231/alntmscore_1_raw_struct_tree.PP.nwk', 'test/HOG1231/alntmscore_0_raw_treescores_struct_tree.json', 'test/HOG1231/alntmscore_0_exp_treescores_struct_tree.json', 'test/HOG1231/alnfident_dist_1_raw_.json', 'test/HOG1231/templateXfoldtree.nx.treefile.rooted', 'test/HOG1231/lddt_distmat.npy', 'test/HOG1231/templateXAA.nx.log', 'test/HOG1231/3di_xaln_tree.PP.nwk.rooted', 'test/HOG1231/logs', 'test/HOG1231/templateXfoldtree.nx.treefile.rooted.final', 'test/HOG1231/3di_xaln_tree.PP.nwk.rooted.final.treescore', 'test/HOG1231/templateX3di.nx.iqtree', 'test/HOG1231/alnfident_dist_3di_AA.json', 'test/HOG1231/lddt_1_exp_struct_tree.PP.nwk', 'test/HOG1231/sequences.aln.muscle.fst.log', 'test/HOG1231/rejected', 'test/HOG1231/fident_0_raw

In [6]:
sample1 = sample_hogs[0]
treeML = sample1+'sequences.aln.muscle.fst.treefile.rooted'
treeFT = sample1+'fident_1_raw_struct_tree.PP.nwk.rooted'
finalset = sample1 +'finalset.csv'

In [7]:
import toytree
import ete3
from ete3 import PhyloTree
from ete3 import NCBITaxa

ncbi = NCBITaxa()
#ncbi.update_taxonomy_database()


In [75]:

import copy

def calc_scores(t , uniprot_csv):
    uniprot_df = pd.read_csv(uniprot_csv)
    tree = toytree.tree(t)
    lineages = treescore.make_lineages(uniprot_df)
    species = treescore.get_species(uniprot_df)
    tree = treescore.label_leaves( tree , lineages , species)
    #tree = treescore.labelwRED(tree.treenode)
    overlap = treescore.getTaxOverlap(tree.treenode)
    taxscore = tree.treenode.score

    #calc descriptive stats on normalized branch lens to see if trees are balanced  
    lengths = np.array([node.dist for node in tree.treenode.traverse()])
    lengths /= np.sum(lengths)
    #calc the root first taxscore

    tree4 = copy.deepcopy(tree)
    treescore.getTaxOverlap_root(tree4.treenode)
    root_score , root_score_nr = treescore.sum_rootscore(tree4.treenode)


    species_set = set()
    #label the leaves with species and number
    for l in tree.treenode.get_leaves():
        if l.sp_num:
            l.name = l.sp_num
            species_set.add(l.sp_num.split('_')[0])


    #get ncbi tree of species set
    species_set = list(species_set)
    species_set = [int(s) for s in species_set]
    species_set = ncbi.get_topology(species_set)

    #change to phylo tree
    ncbitree = PhyloTree(species_set.write() , sp_naming_function=None)

    ncbitree.write(outfile=uniprot_csv.replace('.csv','_ncbi_tree.nwk' ) , format=1)
    etetree = PhyloTree(tree.write() , sp_naming_function=None)

    for l in etetree.get_leaves():
        l.species   = l.name.split('_')[0]
    
    recon_tree, events = etetree.reconcile(ncbitree)
    recon_dups = recon_tree.search_nodes(evoltype="D")
    recon_losses = recon_tree.search_nodes(evoltype="L")
    recon_speciations = recon_tree.search_nodes(evoltype="S")
    print( 'algo 1' )
    print('dups:', len(recon_dups))
    print('losses:', len(recon_losses))
    print('speciations:', len(recon_speciations))

    print( 'algo 2' )
    events = etetree.get_descendant_evol_events()
    dups = etetree.search_nodes(evoltype="D")
    losses = etetree.search_nodes(evoltype="L")
    speciations = etetree.search_nodes(evoltype="S")    
    print('dups:', len(dups))
    print('losses:', len(losses))
    print('speciations:', len(speciations))

    scores = {}
    #measure the distances of leaves to root
    distances = np.array([ node.get_distance(tree.treenode) for node in tree.treenode.get_leaves() ])
    distances_norm = distances / np.mean(distances)
    scores[t] = {'score': taxscore, 'stats': describe(lengths) , 'ultrametricity':  describe(distances), 
                    'ultrametricity_norm':  describe(distances_norm) , 'root_score': root_score , 'root_score_nr': root_score_nr  , 
                    'SO_speciations': len(recon_speciations) , 'SO_dups': len(recon_dups) , 'SO_losses': len(recon_losses) ,
                    'RECON_speciations':speciations ,'RECON_dups': len(dups) , 'RECON_losses': len(losses)  }
    return scores

In [78]:
for sample in sample_hogs:
    print( sample)
    treeML = sample+'sequences.aln.muscle.fst.treefile.rooted'
    treeFT = sample+'fident_1_exp_struct_tree.PP.nwk.rooted'
    finalset = sample +'finalset.csv'

    print('ML')
    scores = calc_scores(treeML , finalset)
    print(scores)
    print('FT')
    scores = calc_scores(treeFT , finalset)
    print(scores)

../OMA_data_unfiltered/OMA_data/LUCA/HOG1077/
ML
root scoring
root score 4623
root score_nr 3909
algo 1
dups: 23
losses: 654
speciations: 35
algo 2
dups: 12
losses: 0
speciations: 46
{'../OMA_data_unfiltered/OMA_data/LUCA/HOG1077/sequences.aln.muscle.fst.treefile.rooted': {'score': 626, 'stats': DescribeResult(nobs=117, minmax=(2.991111384783379e-08, 0.0726053954572658), mean=0.00854700854700855, variance=0.00017982310920735504, skewness=2.671951856873734, kurtosis=7.301067678776121), 'ultrametricity': DescribeResult(nobs=59, minmax=(1.707441299, 4.3434943268), mean=2.139202316576271, variance=0.16713058024887115, skewness=3.168209235115583, kurtosis=13.302032524293164), 'ultrametricity_norm': DescribeResult(nobs=59, minmax=(0.7981672821543633, 2.030427086369106), mean=1.0, variance=0.03652180224239135, skewness=3.1682092351155826, kurtosis=13.302032524293164), 'root_score': 4623, 'root_score_nr': 3909, 'SO_speciations': 35, 'SO_dups': 23, 'SO_losses': 654, 'RECON_speciations': [PhyloT

In [77]:
astralpath = '../A-pro/ASTRAL-MP/astral.1.1.6.jar'

In [11]:
#prepare final set df for astral

In [55]:
import subprocess
#running astral


def run_astral( phylo , st,  mapping, output ,root  , astralpath = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/software/ASTER-Linux/bin/astral-pro' ):

    print('running astral')
    cmd = astralpath + ' -c {st} -a {mapping} -u 2 -i {phylo} -o {output} -C --root {root} 2>{logfile}'.format(phylo = phylo , st=st, mapping = mapping , output = output , root=root,logfile = output+'.log')

    print(cmd)
    subprocess.run(cmd , shell = True)
    return output


def prepare_astral_input(uniprot_df , output):
    finalset = pd.read_csv(uniprot_df)
    finalset['species'] = finalset['Taxonomic lineage (Ids)'].map(lambda x: x.split(',')[-1].split('(')[0].strip())
    mapper = dict(zip(finalset['Entry'], finalset['species']))
    with open(output , 'w') as f:
        for k,v in mapper.items():
            f.write('{}\t{}\n'.format(k,v))
    return mapper , output



In [74]:
from Bio import Phylo
import re

"""

q1,q2,q3: these three values show quartet support (as defined in the description of -t 1) for the main topology, the first alternative, and the second alternative, respectively.
f1, f2, f3: these three values show the total number of quartet trees in all the gene trees that support the main topology, the first alternative, and the second alternative, respectively.
pp1, pp2, pp3: these three show the local posterior probabilities (as defined in the description of -t 4) for the main topology, the first alternative, and the second alternative, respectively.

"""

def parse_astral(string , index = 0):   
    #'[pp1=0.284784;pp2=0.426968;pp3=0.288248;f1=0.000000;f2=0.263158;f3=0.008772;q1=0.000000;q2=0.967742;q3=0.032258]'
    string = string.replace('[','').replace(']','')
    string = string.split(';')
    string = {s.split('=')[0]:float(s.split('=')[1]) for s in string}
    #to dataframe
    string = pd.DataFrame(string , index = [index])
    return string

def retastral_support( astral_file):
    with open(astral_file , 'r') as f:
        nstring = f.read()
    #find all quoted strings with single quotes
    quoted = re.findall(r"'(.*?)'", nstring)
    dfs = [ parse_astral(q,i) for i,q in enumerate(quoted)]
    return pd.concat(dfs)

for sample in sample_hogs:  
    ncbitree = sample+'finalset_ncbi_tree.nwk'
    speciesmap, speciesmap_file = prepare_astral_input(sample+'finalset.csv' , sample+'speciesmap.txt')
    FT = sample+'fident_1_exp_struct_tree.PP.nwk.rooted'
    ML = sample+'sequences.aln.muscle.fst.treefile.rooted'
    print('FT')

    #open ncbi tree
    t = PhyloTree(ncbitree , sp_naming_function=None)
    #find root node name
    root = t.get_tree_root()
    root.name = 'root'
    t.write(outfile=ncbitree , quoted_node_names = False, format=0)

    output = run_astral(  FT , ncbitree,speciesmap_file , sample+'FT_astral_support.tre' , 'root')
    astraldf = retastral_support(output)
    print(astraldf)
    print('ML')
    output = run_astral( ML , ncbitree,speciesmap_file , sample+'ML_astral_support.tre', 'root')
    astraldf = retastral_support(output)
    print(astraldf)



FT
running astral
/work/FAC/FBM/DBC/cdessim2/default/dmoi/software/ASTER-Linux/bin/astral-pro -c ../OMA_data_unfiltered/OMA_data/LUCA/HOG1077/finalset_ncbi_tree.nwk -a ../OMA_data_unfiltered/OMA_data/LUCA/HOG1077/speciesmap.txt -u 2 -i ../OMA_data_unfiltered/OMA_data/LUCA/HOG1077/fident_1_exp_struct_tree.PP.nwk.rooted -o ../OMA_data_unfiltered/OMA_data/LUCA/HOG1077/FT_astral_support.tre -C --root root 2>../OMA_data_unfiltered/OMA_data/LUCA/HOG1077/FT_astral_support.tre.log
        pp1       pp2       pp3        f1        f2        f3        q1  \
0  0.284784  0.426968  0.288248  0.000000  0.263158  0.008772  0.000000   
1  0.357007  0.336824  0.306169  0.100000  0.063158  0.000000  0.612903   
2  0.333333  0.333333  0.333333  0.000000  0.000000  0.000000       NaN   
3  0.345342  0.328480  0.326178  0.035088  0.004386  0.000000  0.888889   
4  0.333333  0.333333  0.333333  0.000000  0.000000  0.000000       NaN   
5  0.320829  0.354424  0.324747  0.000000  0.061776  0.007722  0.000000 